In [3]:
import requests
import json
import pandas as pd
import time
from datetime import datetime, timedelta

def json_prettify(json_res):
    return json.dumps(json_res.json(), indent=2)

def get_auth():
    url = 'http://172.18.65.246/zabbix/api_jsonrpc.php'

    headers = {
        'Content-Type': 'application/json-rpc'
    }
    auth_query = {
        "jsonrpc": "2.0",
        "method": "user.login",
        "params": {
            "user": "QAC",
            "password": "QAC12345"
        },
        "id": 1
    }


    response = requests.post(url, headers=headers, data=json.dumps(auth_query)).json()
    return response['result']
auth = get_auth()

ConnectTimeout: HTTPConnectionPool(host='172.18.65.246', port=80): Max retries exceeded with url: /zabbix/api_jsonrpc.php (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000254908E3F50>, 'Connection to 172.18.65.246 timed out. (connect timeout=None)'))

In [ ]:

def to_timestamp(unix_timestamp):
    # Convert Unix timestamp to a datetime object
    timestamp = datetime.utcfromtimestamp(int(unix_timestamp))
    timestamp = timestamp + timedelta(hours=7)
    # Format the datetime object as a human-readable timestamp
    formatted_timestamp = timestamp.strftime('%Y-%m-%d 00:00:00')
    # formatted_timestamp = timestamp.strftime('%Y-%m-%d %H:%M:%S')

    return formatted_timestamp



def datetime_to_unix(start_date, end_date):

    # Convert datetime objects to Unix timestamps
    start_unix = int(start_date.timestamp())
    end_unix = int((end_date).timestamp())

    # Add 7 hours to the Unix timestamps
    start_unix = start_unix + 7 * 3600
    end_unix = end_unix + 7 * 3600
    return start_unix, end_unix

def get_data(auth, time_start, time_till, item_id):
    time_start = int(datetime.strptime(time_start, "%Y-%m-%d %H:%M:%S").timestamp())
    time_till = int(datetime.strptime(time_till, "%Y-%m-%d %H:%M:%S").timestamp())

    url = 'http://172.18.65.246/zabbix/api_jsonrpc.php'
    headers = {
        'Content-Type': 'application/json-rpc',
    }

    data_query = {
            "jsonrpc": "2.0",
            "method": "history.get",
            "params": {
                "output": "extend",
                "history": 0,
                "itemids": item_id,
                "sortfield": "clock",
                "sortorder": "ASC",
                "time_from": time_start,
                "time_till": time_till,
                "limit": 50000
            },
            "auth": auth,
            "id": 1
        }

    response = requests.post(url, headers=headers, data=json.dumps(data_query)).json()
    
    df = pd.DataFrame(response['result'])

    return df

In [ ]:
df = pd.read_excel('CPL - Export Data Zabbix DCI.xlsx', index_col=None)

In [11]:
date_start = "2023-11-01 00:00:00"
date_end = "2023-11-30 23:59:59"

base_dir = './raw_data/Nov 2023/'

df_agg = pd.DataFrame()
host_no_data = []

timestamp_list = pd.date_range(start=date_start, end=date_end, freq='D').strftime("%Y-%m-%d %H:%M:%S")
for index, data in df[:].iterrows():
    hostname = data['Zabbix Host']
    item_id = data['itemid'] 
    for date_time in timestamp_list.to_list():
        datetime_start = str(date_time[0:10]+" 00:00:00")
        datetime_end = str(date_time[0:10]+ " 23:59:59")
        # datetime_start = datetime.strptime(str(date_time[0:10]+" 00:00:00"), "%Y-%m-%d %H:%M:%S")
        # datetime_end = datetime.strptime(str(date_time[0:10]+ " 23:59:59"), "%Y-%m-%d %H:%M:%S")
        raw_data = get_data(auth, datetime_start, datetime_end, item_id)

        if len(raw_data) > 0:
            raw_data['date'] = raw_data['clock'].apply(to_timestamp)
            raw_data['value'] = raw_data['value'].astype(float).astype(int)
            ds = date_time[0:10].replace('-', "")
            # Create unix time
            # raw_data_daily = raw_data[raw_data['date'] == date_time]

            data_max = {"date": str(date_time), "hostname":hostname, "metrics":"max", "value":raw_data['value'].max(), "ds": ds}
            data_min = {"date": str(date_time), "hostname":hostname, "metrics":"min", "value":raw_data['value'].min(), "ds": ds}
            data_mean = {"date": str(date_time), "hostname":hostname, "metrics":"mean", "value":raw_data['value'].mean() , "ds": ds}
            data_median = { "date": str(date_time), "hostname":hostname, "metrics":"median", "value":raw_data['value'].median(), "ds": ds}

            df_agg = pd.concat([df_agg, pd.DataFrame([data_max, data_min,data_mean,data_median])], ignore_index=True)
        
            # datetime_start = datetime_start + timedelta(days=1)
        else:
            host_no_data.append(hostname)
            continue
    print('hostname: '+hostname)
df_agg.to_csv(str(base_dir+'Zabbix_CPL_agg (Nov 2023).csv'), index=False)

hostname: GTI_GU21_UPSA1
hostname: GTI_GU21_UPSA2
hostname: GTI_GU21_UPSA3
hostname: GTI_GU21_UPSB1
hostname: GTI_GU21_UPSB2
hostname: GTI_GU21_UPSB3
hostname: GTI_GU41_UPSA1
hostname: GTI_GU41_UPSA2
hostname: GTI_GU42_UPSB1
hostname: GTI_GU42_UPSB2
hostname: GA31_DBPAC_A
hostname: GA31_DBPAC_B
hostname: GU41_DBPAC_A
hostname: GU41_DBPAC_B


In [14]:
df_agg.to_csv(str(base_dir+'Zabbix_CPL_agg (Nov 2023).csv'), index=False)